***GENERATED CODE FOR sbaloanapproval PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Name', 'transformation_label': 'String Indexer'}], 'feature': 'Name', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '500', 'mean': '', 'stddev': '', 'min': '02R3125594005ION OF AMERICA', 'max': 'YALE MATERIALS HANDLING-ALABAM', 'missing': '0', 'distinct': '493'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Name'}, {'feature_label': 'Name', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Name')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'City', 'transformation_label': 'String Indexer'}], 'feature': 'City', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'AKRON', 'max': 'York', 'missing': '0', 'distinct': '405'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'City'}, {'feature_label': 'City', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('City')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'State', 'transformation_label': 'String Indexer'}], 'feature': 'State', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'AK', 'max': 'WY', 'missing': '0', 'distinct': '46'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'State'}, {'feature_label': 'State', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('State')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Zip', 'transformation_label': 'String Indexer'}], 'feature': 'Zip', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '47349.9', 'stddev': '30500.12', 'min': '0', 'max': '99999', 'missing': '0', 'distinct': '438'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Zip'}, {'feature_label': 'Zip', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Zip')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Bank', 'transformation_label': 'String Indexer'}], 'feature': 'Bank', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '499', 'mean': '', 'stddev': '', 'min': 'AMERICA FIRST FCU', 'max': 'ZIONS FIRST NATIONAL BANK', 'missing': '1', 'distinct': '151'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Bank'}, {'feature_label': 'Bank', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Bank')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'BankState', 'transformation_label': 'String Indexer'}], 'feature': 'BankState', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '499', 'mean': '', 'stddev': '', 'min': 'AL', 'max': 'WY', 'missing': '1', 'distinct': '50'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'BankState'}, {'feature_label': 'BankState', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('BankState')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'NAICS', 'transformation_label': 'String Indexer'}], 'feature': 'NAICS', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '348120.38', 'stddev': '272713.62', 'min': '0', 'max': '813910', 'missing': '0', 'distinct': '228'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'NAICS'}, {'feature_label': 'NAICS', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('NAICS')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ApprovalDate', 'transformation_label': 'String Indexer'}], 'feature': 'ApprovalDate', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '1-Mar-84', 'max': '9-Feb-98', 'missing': '0', 'distinct': '10'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ApprovalDate'}, {'feature_label': 'ApprovalDate', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ApprovalDate')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ApprovalFY', 'transformation_label': 'String Indexer'}], 'feature': 'ApprovalFY', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '2002.62', 'stddev': '5.23', 'min': '1984', 'max': '2006', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ApprovalFY'}, {'feature_label': 'ApprovalFY', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ApprovalFY')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'RevLineCr', 'transformation_label': 'String Indexer'}], 'feature': 'RevLineCr', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '0.0', 'stddev': '0.0', 'min': '0', 'max': 'Y', 'missing': '0', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'RevLineCr'}, {'feature_label': 'RevLineCr', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('RevLineCr')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'LowDoc', 'transformation_label': 'String Indexer'}], 'feature': 'LowDoc', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'N', 'max': 'Y', 'missing': '2', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'LowDoc'}, {'feature_label': 'LowDoc', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('LowDoc')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ChgOffDate', 'transformation_label': 'String Indexer'}], 'feature': 'ChgOffDate', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '199', 'mean': '', 'stddev': '', 'min': '1-Dec-11', 'max': 'N', 'missing': '352', 'distinct': '180'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ChgOffDate'}, {'feature_label': 'ChgOffDate', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ChgOffDate')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'DisbursementDate', 'transformation_label': 'String Indexer'}], 'feature': 'DisbursementDate', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '10-Aug-98', 'max': '9-Sep-98', 'missing': '1', 'distinct': '98'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'DisbursementDate'}, {'feature_label': 'DisbursementDate', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('DisbursementDate')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'DisbursementGross', 'transformation_label': 'String Indexer'}], 'feature': 'DisbursementGross', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '$1,000,000.00 ', 'max': '$987.00 ', 'missing': '0', 'distinct': '284'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'DisbursementGross'}, {'feature_label': 'DisbursementGross', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('DisbursementGross')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'BalanceGross', 'transformation_label': 'String Indexer'}], 'feature': 'BalanceGross', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '$0.00 ', 'max': '$0.00 ', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'BalanceGross'}, {'feature_label': 'BalanceGross', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('BalanceGross')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'MIS_Status', 'transformation_label': 'String Indexer'}], 'feature': 'MIS_Status', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '499', 'mean': '', 'stddev': '', 'min': '$0.00 ', 'max': 'P I F', 'missing': '1', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'MIS_Status'}, {'feature_label': 'MIS_Status', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('MIS_Status')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ChgOffPrinGr', 'transformation_label': 'String Indexer'}], 'feature': 'ChgOffPrinGr', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '$0.00 ', 'max': '$98,931.00 ', 'missing': '0', 'distinct': '57'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ChgOffPrinGr'}, {'feature_label': 'ChgOffPrinGr', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ChgOffPrinGr')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'GrAppv', 'transformation_label': 'String Indexer'}], 'feature': 'GrAppv', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '$1,000,000.00 ', 'max': '$990,000.00 ', 'missing': '0', 'distinct': '233'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'GrAppv'}, {'feature_label': 'GrAppv', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('GrAppv')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'SBA_Appv', 'transformation_label': 'String Indexer'}], 'feature': 'SBA_Appv', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '$1,012,500.00 ', 'max': '$937,500.00 ', 'missing': '0', 'distinct': '298'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'SBA_Appv'}, {'feature_label': 'SBA_Appv', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('SBA_Appv')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15, use_dask=True)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run sbaloanapprovalHooks.ipynb
try:
	#sourcePreExecutionHook()

	sbaloanapproval = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/SBA loan approval.csv', 'filename': 'SBA loan approval.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'domain': 'http://172.31.59.158', 'port': '40070', 'dirPath': '/FileStore/platform', 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/'}")
	#sourcePostExecutionHook(sbaloanapproval)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run sbaloanapprovalHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(sbaloanapproval,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "LoanNr_ChkDgt", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1833054090.99", "stddev": "990665.24", "min": "1831475009", "max": "1834725007", "missing": "0"}, "updatedLabel": "LoanNr_ChkDgt"}, {"transformationsData": [{"feature_label": "Name", "transformation_label": "String Indexer"}], "feature": "Name", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "02R3125594005ION OF AMERICA", "max": "YALE MATERIALS HANDLING-ALABAM", "missing": "0", "distinct": "493"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Name"}, {"transformationsData": [{"feature_label": "City", "transformation_label": "String Indexer"}], "feature": "City", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "AKRON", "max": "York", "missing": "0", "distinct": "405"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "City"}, {"transformationsData": [{"feature_label": "State", "transformation_label": "String Indexer"}], "feature": "State", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "AK", "max": "WY", "missing": "0", "distinct": "46"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "State"}, {"transformationsData": [{"feature_label": "Zip", "transformation_label": "String Indexer"}], "feature": "Zip", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "47349.9", "stddev": "30500.12", "min": "0", "max": "99999", "missing": "0", "distinct": "438"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Zip"}, {"transformationsData": [{"feature_label": "Bank", "transformation_label": "String Indexer"}], "feature": "Bank", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "499", "mean": "", "stddev": "", "min": "AMERICA FIRST FCU", "max": "ZIONS FIRST NATIONAL BANK", "missing": "1", "distinct": "151"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Bank"}, {"transformationsData": [{"feature_label": "BankState", "transformation_label": "String Indexer"}], "feature": "BankState", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "499", "mean": "", "stddev": "", "min": "AL", "max": "WY", "missing": "1", "distinct": "50"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "BankState"}, {"transformationsData": [{"feature_label": "NAICS", "transformation_label": "String Indexer"}], "feature": "NAICS", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "348120.38", "stddev": "272713.62", "min": "0", "max": "813910", "missing": "0", "distinct": "228"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "NAICS"}, {"transformationsData": [{"feature_label": "ApprovalDate", "transformation_label": "String Indexer"}], "feature": "ApprovalDate", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "1-Mar-84", "max": "9-Feb-98", "missing": "0", "distinct": "10"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ApprovalDate"}, {"transformationsData": [{"feature_label": "ApprovalFY", "transformation_label": "String Indexer"}], "feature": "ApprovalFY", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "2002.62", "stddev": "5.23", "min": "1984", "max": "2006", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ApprovalFY"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Term", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "98.9", "stddev": "72.6", "min": "0", "max": "306", "missing": "0"}, "updatedLabel": "Term"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "NoEmp", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "8.03", "stddev": "13.49", "min": "0", "max": "115", "missing": "0"}, "updatedLabel": "NoEmp"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "NewExist", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1.28", "stddev": "0.45", "min": "1", "max": "2", "missing": "0"}, "updatedLabel": "NewExist"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "CreateJob", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1.93", "stddev": "6.56", "min": "0", "max": "70", "missing": "0"}, "updatedLabel": "CreateJob"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "RetainedJob", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "5.13", "stddev": "13.36", "min": "0", "max": "126", "missing": "0"}, "updatedLabel": "RetainedJob"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "FranchiseCode", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1568.54", "stddev": "9834.61", "min": "0", "max": "75917", "missing": "0"}, "updatedLabel": "FranchiseCode"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "UrbanRural", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.73", "stddev": "0.71", "min": "0", "max": "2", "missing": "0"}, "updatedLabel": "UrbanRural"}, {"transformationsData": [{"feature_label": "RevLineCr", "transformation_label": "String Indexer"}], "feature": "RevLineCr", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "Y", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "RevLineCr"}, {"transformationsData": [{"feature_label": "LowDoc", "transformation_label": "String Indexer"}], "feature": "LowDoc", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "N", "max": "Y", "missing": "2", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "LowDoc"}, {"transformationsData": [{"feature_label": "ChgOffDate", "transformation_label": "String Indexer"}], "feature": "ChgOffDate", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "199", "mean": "", "stddev": "", "min": "1-Dec-11", "max": "N", "missing": "352", "distinct": "180"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ChgOffDate"}, {"transformationsData": [{"feature_label": "DisbursementDate", "transformation_label": "String Indexer"}], "feature": "DisbursementDate", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "10-Aug-98", "max": "9-Sep-98", "missing": "1", "distinct": "98"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "DisbursementDate"}, {"transformationsData": [{"feature_label": "DisbursementGross", "transformation_label": "String Indexer"}], "feature": "DisbursementGross", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "$1,000,000.00 ", "max": "$987.00 ", "missing": "0", "distinct": "284"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "DisbursementGross"}, {"transformationsData": [{"feature_label": "BalanceGross", "transformation_label": "String Indexer"}], "feature": "BalanceGross", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "$0.00 ", "max": "$0.00 ", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "BalanceGross"}, {"transformationsData": [{"feature_label": "MIS_Status", "transformation_label": "String Indexer"}], "feature": "MIS_Status", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "499", "mean": "", "stddev": "", "min": "$0.00 ", "max": "P I F", "missing": "1", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "MIS_Status"}, {"transformationsData": [{"feature_label": "ChgOffPrinGr", "transformation_label": "String Indexer"}], "feature": "ChgOffPrinGr", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "$0.00 ", "max": "$98,931.00 ", "missing": "0", "distinct": "57"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ChgOffPrinGr"}, {"transformationsData": [{"feature_label": "GrAppv", "transformation_label": "String Indexer"}], "feature": "GrAppv", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "$1,000,000.00 ", "max": "$990,000.00 ", "missing": "0", "distinct": "233"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "GrAppv"}, {"transformationsData": [{"feature_label": "SBA_Appv", "transformation_label": "String Indexer"}], "feature": "SBA_Appv", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "$1,012,500.00 ", "max": "$937,500.00 ", "missing": "0", "distinct": "298"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "SBA_Appv"}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run sbaloanapprovalHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionClassification(autofe, ["LoanNr_ChkDgt", "Term", "NoEmp", "NewExist", "CreateJob", "RetainedJob", "FranchiseCode", "UrbanRural", "Name_stringindexer", "City_stringindexer", "State_stringindexer", "Zip_stringindexer", "Bank_stringindexer", "BankState_stringindexer", "NAICS_stringindexer", "ApprovalDate_stringindexer", "ApprovalFY_stringindexer", "RevLineCr_stringindexer", "LowDoc_stringindexer", "ChgOffDate_stringindexer", "DisbursementDate_stringindexer", "DisbursementGross_stringindexer", "BalanceGross_stringindexer", "ChgOffPrinGr_stringindexer", "GrAppv_stringindexer", "SBA_Appv_stringindexer"], "MIS_Status_stringindexer")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

spark.stop()

